In [ ]:
import pandas as pd
df = pd.read_csv("data/events.csv", parse_dates=["timestamp"])

view_sessions = df[df.event=="view"].session_id.nunique()
add_sessions = df[df.event=="add_to_cart"].session_id.nunique()
checkout_sessions = df[df.event=="checkout_start"].session_id.nunique()
purchase_sessions = df[df.event=="purchase"].session_id.nunique()

print("Sessions: views", view_sessions, "adds", add_sessions, "checkouts", checkout_sessions, "purchases", purchase_sessions)

conv_view_to_purchase = purchase_sessions / view_sessions
conv_add_to_purchase = purchase_sessions / add_sessions if add_sessions else 0
aov = df[df.event=="purchase"].order_value.mean()

print(f"Conversion view→purchase: {conv_view_to_purchase:.2%}")
print(f"AOV: ₹{aov:.2f}")


In [ ]:
def funnel_by(group_col):
    g = df.groupby([group_col, "event"]).session_id.nunique().unstack(fill_value=0)
    g["view_to_add"] = g.get("add_to_cart",0) / g.get("view",1)
    g["add_to_checkout"] = g.get("checkout_start",0) / g.get("add_to_cart",1)
    g["checkout_to_purchase"] = g.get("purchase",0) / g.get("checkout_start",1)
    g["view_to_purchase"] = g.get("purchase",0) / g.get("view",1)
    return g

funnel_by("city_tier")
